In [ ]:
import os
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error, roc_auc_score
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
import xgboost as xgb


import warnings
warnings.filterwarnings('ignore')

In [ ]:
PATH = '../input/tabular-playground-series-nov-2021'
train = pd.read_csv(os.path.join(PATH, 'train.csv'))
test = pd.read_csv(os.path.join(PATH, 'test.csv'))
sub = pd.read_csv(os.path.join(PATH, 'sample_submission.csv'))

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['target'].value_counts(normalize=True)

In [ ]:
def random_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
columns = [col for col in train.columns if col not in ['id', 'target'] ]

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

In [ ]:
SEED = 2001
N_FOLDS =20

random_seed(SEED)

auc_score = []

train = train.sample(frac=1).reset_index(drop=True)

targets = train['target'].values

kf = StratifiedKFold(n_splits = N_FOLDS, shuffle=True, random_state=SEED)    
        
oof = np.zeros((train.shape[0],))
test_preds = 0

for f, (train_idx, val_idx) in tqdm(enumerate(kf.split(train, targets))):
        df_train, df_val = train.iloc[train_idx][columns], train.iloc[val_idx][columns]
        train_target, val_target = targets[train_idx], targets[val_idx]
        
        model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=SEED,
            n_jobs=-1
        )
        
        model.fit(df_train[columns], train_target)
        
        oof_tmp = model.predict_proba(df_val[columns])[:,1]
        test_tmp = model.predict_proba(test[columns])[:,1]   
        
        oof[val_idx] = oof_tmp
        test_preds += test_tmp/N_FOLDS
        auc = roc_auc_score(val_target, oof_tmp)
        auc_score.append(auc)
        print(f'FOLD: {f} AUC: {auc} Mean AUC: {np.mean(auc_score)}')

In [ ]:
sub['target'] = test_preds
sub.to_csv('submission.csv', index=False)